1. 시작 타임스탬프 찍기
2. 드라이버 호출 + 헤더 옵션 지정 + 전체화면 지정
3. 드라이버 다음 웹페이지 접속
4. 5초 대기 
5. while반복문 - 페이지 전환용 1
6. 전체 페이지 파싱 - 변수에 담기
7. 전체 페이지에서 전체 ul태그 수집하기 
8. ul태그에서 각 뉴스 제목 리스트에 담기 
9. for반복문 돌면서 뉴스 제목태그 별 링크 수집해서 url_list에 리스트에 추가하기 
10. 하나 수집할 때 마다 no+=1 (초기값1)
11. If no > cnt : for문 break
12. If no > cnt : while문 break
13. If page_cnt == 2 : 
while문 break
1. 다음페이지 xpath로 click(), 페이지 전환하기 
2. time.sleep(5)
3. page_cnt +=1 (기본값 = 1)


1. for문으로 url_list돌면서 드라이버 호출, 하나하나 접속하기 
2. 각 페이지 접속해서 페이지 파싱
3. 드라이버.quit()
4. 언론사 제목, 기사 제목, 부제목 수집 
5. 각 수집한 텍스트 데이터 리스트에 추가

1. F.close()
2. 데이터프레임 생성
3. 각 데이터프레임에 [‘언론사’], [‘기사제목’], 
[‘요약’]  칼럼 생성하고 리스트 넣기 
4. 데이터프레임 csv, xlsx형식으로 저장하기 
5. 종료 타임스탬프 찍기
6. 전체 소요시간 측정하기 
7. Cnt, len(url_list)로 요청건수, 실제 수집 건수 사용자에게 출력해서 보여주기 
8. 전체 소요시간 사용자에게 보여주기 
끝




In [1]:
import selenium
from selenium import webdriver
import os
import time
from bs4 import BeautifulSoup
import pandas as pd

import openpyxl
from openpyxl import load_workbook
from openpyxl import Workbook
from PIL import Image
from io import BytesIO
import requests

In [2]:
print('-'*100)
print('스트레이트의 포털 뉴스 노출 보도, 사실일까? 확인해보자')
print('='*100)

cnt = int(input('1.총 몇 건의 뉴스 기사 데이터를 수집하시겠습니까?(최대 30건) : '))
f_dir = input('2.파일 저장경로를 입력하세요 : ')
if f_dir == "" : 
    f_dir = '/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'

now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+'포털 언론 기사 노출')
os.chdir(f_dir+s+'-'+'포털 언론 기사 노출')

f_name = s+'-'+'포털 언론 기사 노출'+'.txt'
fc_name = s+'-'+'포털 언론 기사 노출'+'.csv'
fx_name = s+'-'+'포털 언론 기사 노출'+'.xlsx'

----------------------------------------------------------------------------------------------------
스트레이트의 포털 뉴스 노출 보도, 사실일까? 확인해보자
1.총 몇 건의 뉴스 기사 데이터를 수집하시겠습니까?(최대 30건) : 30
2.파일 저장경로를 입력하세요 : 


In [3]:
#1. 시작 타임스탬프 찍기
s_time = time.time()

#드라이버 호출, 옵션 지정
path = '/Users/kibeomkim/Desktop/Chromedriver_88/chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15')
#options.add_argument('--kiosk')
driver = webdriver.Chrome(path, options=options)
driver.get('https://www.daum.net/?nil_profile=mini&nil_src=daum')
time.sleep(5)

In [4]:
#페이지 전환용 while 반복문 사용하기 
page_cnt = 1
no = 1
url_list = []
while True : 
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    news_list = soup.find('ul', 'list_txt').find_all('a')
    
    #for 반복문 돌면서 링크 수집하기 
    for a in news_list : 
        text = a.get_text().replace('\n', "").strip()
        print(text)
        url = a['href']
        url_list.append(url)
        no += 1
        
        if no > cnt : 
            break
    if no > cnt : 
        break
    elif page_cnt == 2 : 
        break
    driver.find_element_by_xpath('/html/body/div[2]/main/div[1]/article/div[2]/div[1]/div/div[8]/button[2]').click()
    page_cnt += 1
    time.sleep(1)

driver.quit()

f = open(f_name, 'a', encoding='utf-8')
img_list1 = []
press_name1 = []
title_list1= []
sum1_list1 = []


for b in url_list : 
    path = '/Users/kibeomkim/Desktop/Chromedriver_88/chromedriver'
    options = webdriver.ChromeOptions()
    options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15')
    #options.add_argument('--kiosk')
    driver = webdriver.Chrome(path, options=options)
    try : 
        driver.get(b)
        time.sleep(5)
    except : 
        driver.get('http:'+b)
        time.sleep(5)
    
    #접속 후 페이지 파싱
    html1 = driver.page_source
    soup1 = BeautifulSoup(html1, 'html.parser')
    driver.quit()
    
    img_url = soup1.find('em', 'info_cp').find('img', 'thumb_g')['src']
    img_list1.append(img_url)
    
    f.write('-'*100+'\n')
    print('-'*100)
    
    press_name = soup1.find('em', 'info_cp').find('img', 'thumb_g')['alt']
    press_name1.append(press_name)
    f.write('1.언론사 이름 : %s' %press_name+'\n')
    print('1.언론사 이름 : %s' %press_name)
    
    news_title = soup1.find('h3', 'tit_view').get_text().replace('\n', "").strip()
    title_list1.append(news_title)
    f.write('2.뉴스 제목 : %s' %news_title+'\n')
    print('2.뉴스 제목 : %s' %news_title)
    
    try : 
        sum1 = soup1.find('strong', 'summary_view').get_text().replace('\n', "").strip()
        sum1_list1.append(sum1)
        f.write('3.내용 요약 : %s' %sum1+'\n')
        print('3.내용 요약 : %s' %sum1)
    except : 
        f.write('3.내용 요약 : 제공된 정보 없음'+'\n')
        print('3.내용 요약 : 제공된 정보 없음')
        sum1_list1.append(' ')
    
    f.write('-'*100+'\n')
    print('-'*100)
    print()
    
f.close()
news_press = pd.DataFrame()
news_press['언론사'] = press_name1
news_press['기사제목'] = title_list1
news_press['요약'] = sum1_list1
news_press.to_csv(fc_name, encoding='utf-8-sig')
news_press.to_excel(fx_name)

def pic_to_excel() : 
    wb = openpyxl.load_workbook(fx_name)
    ws = wb.worksheets[0]

    for a in range(2, no+1) : 
        ws.row_dimensions[a].height =  80
    
    ws.column_dimensions['B'].width =30


    nor = 2
    for a in img_list1 : 
        try : 
            res = requests.get(a)
        except : 
            res = requests.get('http:'+a)
        image_data = BytesIO(res.content)
        img = openpyxl.drawing.image.Image(image_data)
        ws.add_image(img, anchor='B%s' %nor)
        nor +=1
    wb.save(fx_name)
    print('엑셀에 사진 첨부 완료') 

pic_to_excel()


e_time = time.time()
total = e_time - s_time

print('-'*100)
print('요청하신 총 %s 건 데이터 중에서 %s 건 만큼 수집했습니다' %(cnt, len(url_list)))
print('총 소요시간은 %s 초 입니다' %round(total,2))
print('='*100)

    
    
    
    
    
    
    
    

권익위 "공공기관 감사담당자, 이해 사건 감사업무 배제"
외교부, 한·아프리카연합 장관급 포럼 서울 개최 방안 협의
미 무역대표부 대표 "동맹국들과 연합전선으로 중국 상대"
통일부 "2032 올림픽 우선 협상지 호주 선정..좋은 여건 아냐"
"텍사스 주민 전력시장 경쟁 도입에 요금만 31조원 더 내"
경찰, '배터리 기술유출 의혹' SK이노베이션 추가 압색
완벽한 살인용의자 A순경? 우연에 담긴 진실[탐정 손수호]
"주민이 구상하고 실행"..충북도 20곳서 행복마을사업 추진
김종인 "여론조사 역선택? 너무 자신없어하지 말아야"
주유엔 중국대사 "서방국 자국 인권문제나 해결해라"
광주 명진고 또 '보복 해임 교사 왕따' 불거져
서해 참조기가 안 잡힌다는데..이제는 '양식'으로 해결?
서대문구 백신 접종 첫 대상자는 '요양센터장'..3월까지 7500여명 접종
올해 벚꽃축제도 줄줄이 취소..지자체 대책 고심
"뉴욕서 남아공발 변이 유사 바이러스 급속도로 확산"
미궁에 빠진 '국내 1호 접종자'.. 이유는?
'을왕리 참변' 음주운전자 징역 10년·동승자 6년 구형
기네스 펠트로의 '코로나 극복' 식단..英의료진은 경고했다
"누가 책임지려 하겠나"..임기말 '복지부동' 공무원
물러날 곳 없는 尹의 배수진.. 與 vs 檢 '수사청' 정면충돌하나
김남국·윤미향 등 與의원 "北 강력반발..한미훈련 연기하라"
합의금 3조원 또는 1천억원?..LG와 SK의 '배터리 전쟁 2라운드' 누가 이길까
보상비 10조-GTX-주민반발..광명시흥 신도시 앞 '3개의 산'
"굶기고 고문하고"..싱가포르 미얀마인 가사도우미 끔찍한 죽음
"텍사스 주민 전력시장 경쟁 도입에 요금만 31조원 더 내"
국회서 눈물짓던 홍남기, 어쩌다 '정말 나쁜 사람'이 됐나
'봉오동 전투' 홍범도 장군 부인·아들, 3·1절에 건국훈장 받는다
바람에 깎인 화성의 바위 단면..퍼시비런스의 강력 '줌' 카메라
'용돈 벌려다 철창신세' 연예인 딥페이크 영상 판매 10대 구속
"새 경쟁자 몰려드는 결제산업..카드업계에 '지

----------------------------------------------------------------------------------------------------
1.언론사 이름 : 연합뉴스
2.뉴스 제목 : "텍사스 주민 전력시장 경쟁 도입에 요금만 31조원 더 내"
3.내용 요약 : 제공된 정보 없음
----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
1.언론사 이름 : 시사저널
2.뉴스 제목 : 국회서 눈물짓던 홍남기, 어쩌다 '정말 나쁜 사람'이 됐나
3.내용 요약 : 이낙연, '재난지원금' 공방에 "나쁜 사람" 맹공누적된 與와의 갈등..사퇴 수순 밟을지 '촉각'
----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
1.언론사 이름 : 한국일보
2.뉴스 제목 : '봉오동 전투' 홍범도 장군 부인·아들, 3·1절에 건국훈장 받는다
3.내용 요약 : 제공된 정보 없음
----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
1.언